In [2]:
pip install -U "huggingface_hub"

  Using cached huggingface_hub-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.28.0-py3-none-any.whl (464 kB)
Note: you may need to restart the kernel to use updated packages.


# Download the model files from HuggingFace to Local

In [9]:
from huggingface_hub import snapshot_download
import os
os.makedirs("model", exist_ok=True)
locashun = snapshot_download(repo_id="deepseek-ai/DeepSeek-R1-Distill-Llama-70B", local_dir = "model")

Fetching 26 files:   0%|          | 0/26 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/18.6k [00:00<?, ?B/s]

model-00001-of-000017.safetensors:   0%|          | 0.00/8.95G [00:00<?, ?B/s]

...

model-00014-of-000017.safetensors:   0%|          | 0.00/8.69G [00:00<?, ?B/s]

model-00017-of-000017.safetensors:   0%|          | 0.00/10.5G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/60.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

## Upload the model files from local to s3

In [ ]:
s3_destination = "s3://BUCKET/DIR_PATH/"  ### The directory path in s3 where the model will be uploaded to

In [ ]:
!aws s3 sync {locashun}/ {s3_destination}

# Create a Bedrock Custom Model Import 

In [3]:
import boto3
import json
import sagemaker
role = sagemaker.get_execution_role()
region = "us-east-1"
bedrock = boto3.client(service_name='bedrock', region_name=region)

job_name = "r1-distill-llama70b" 

imported_model_name = "r1-distill-llama-70b"
model_s3_uri = s3_destination

# createModelImportJob API
create_job_response = bedrock.create_model_import_job(
    jobName=job_name,
    importedModelName=imported_model_name,
    roleArn=role,
    modelDataSource={
        "s3DataSource": {
            "s3Uri": model_s3_uri
        }
    },
)
job_arn = create_job_response.get("jobArn")



In [6]:
# Bedrock custom model import can take a few minutes (~20 mins), lets poll the job until the status is "completed"
import time
while True:
    response = bedrock.get_model_import_job(jobIdentifier=job_arn)
    status = response['status'].upper()
    print(f"Status: {status}")
    
    if status in ['COMPLETED', 'FAILED']:
        break
        
    time.sleep(60)  # Check every 60 seconds

# Get the model ID
model_id = response['importedModelArn']

Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: INPROGRESS
Status: COMPLETED


In [12]:
import boto3
import json

client = boto3.client("bedrock-runtime", region_name=region)

def cmi_query(system_prompt, user_question, max_retries=10, use_reasoning_prompt=False):
    """
    reasoning prompt uses the recommended prompt template and includes the complete thinking step (<think></think>) in its response, however, this incures more latency. This is based on the tokenizer for the model.
    The non-reasoning prompt seldom provides the reasoning step but is faster in response. Basd on the use case select the one that best works for you.
    """
    if not use_reasoning_prompt:
        formatted_prompt = f"""<｜begin▁of▁sentence｜><|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{user_question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"""
    
    else:
        user_question = f"{system_prompt}\n\n{user_question}"
        formatted_prompt = f"""<｜begin▁of▁sentence｜><｜User｜>{user_question}<｜Assistant｜>"""

    native_request = {
        "prompt": formatted_prompt,
        "max_gen_len": 500,
        "top_p": 0.9,
        "temperature": 0.5
    }

    attempt = 0
    while attempt < max_retries:
        try:
            response = client.invoke_model(modelId=model_id, body=json.dumps(native_request))
            response_body = json.loads(response.get('body').read())
            if 'generation' in response_body:
                response_text = response_body['generation'].strip()
                break
        except Excpetion as e:
            print(e)
            print("Model does not appear to be ready. Retrying.")
            attempt += 1
            time.sleep(30)
        

    return response_text or ""

In [17]:
%%time
cmi_query("You are a AI assitant", "how many planets are in our solar system", max_retries=10, use_reasoning_prompt=True)

CPU times: user 5.13 ms, sys: 267 μs, total: 5.39 ms
Wall time: 8.45 s


"<think>\nAlright, so I need to figure out how many planets are in our solar system. I remember learning about this in school, but I'm a bit fuzzy on the details now. Let me start by recalling what I know.\n\nFirst, I think there are eight planets. But wait, didn't Pluto used to be considered a planet? I remember hearing that it was reclassified. So, maybe it's not counted anymore. That would make it eight instead of nine.\n\nLet me list them out to make sure. Starting from the Sun and moving outward, the planets are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. That's eight. Pluto is no longer classified as a planet, right? It's now a dwarf planet. So, the official count is eight.\n\nI should double-check if there are any other planets that I might have missed. Sometimes, I get confused with the order, but I think that's all. Mercury is closest to the Sun, then Venus, Earth, Mars, and then the gas giants: Jupiter, Saturn, Uranus, and Neptune. Yeah, that's eight.\n